In [1]:
import opendatasets as od
import pandas as pd
import time 
from pathlib import Path

In [3]:
dataset_path = Path('imdb-user-reviews', 'song_lyrics.csv')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/datasets/sadmadlad/imdb-user-reviews')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  kseniaviller


Your Kaggle Key:

  ········


Dataset URL: https://www.kaggle.com/datasets/sadmadlad/imdb-user-reviews


100%|██████████████████████████████████████| 15.1M/15.1M [00:01<00:00, 8.28MB/s]


In [5]:
import json


n, mean, M2 = 0, 0.0, 0
for path in Path('imdb-user-reviews').glob('**/*'):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        score = float(info['movieIMDbRating'])
        n += 1
        delta = score - mean
        mean += delta / n
        M2 += delta * (score - mean)

print(mean, (M2 / n) ** (1/2))

8.030000000000001 1.0517128885774867


In [7]:
from functools import reduce

In [61]:
def mapper(path):
    """
    Mapper function that reads a file and returns a tuple with partial results for ratings.
    Returns (n, mean, M2), where:
    - n: number of ratings
    - mean: average of ratings
    - M2: intermediate value for variance calculation
    """
    if path.is_file():
        try:
            with open(path, 'r') as file:
                data = json.load(file)

            # Debugging: Print the path and data
            print(f"Processing file: {path}")
            print(f"Data: {data}")

            rating = data.get('movieIMDbRating')
            if rating is not None:
                print(f"Found rating: {rating}")
                n = 1
                mean = rating
                M2 = 0
                return (n, mean, M2)
            else:
                print(f"No rating found in file: {path}")
        except (json.JSONDecodeError, KeyError, TypeError) as e:
            print(f"Skipping file {path} due to error: {e}")

    return (0, 0, 0)


In [63]:
def reducer(score_data1, score_data2):
    """
    Reducer function that combines two data parts and returns updated n, mean, and M2.
    """
    n1, mean1, M2_1 = score_data1
    n2, mean2, M2_2 = score_data2

    n = n1 + n2
    if n == 0:
        return (0, 0, 0)

    delta = mean2 - mean1
    mean = mean1 + delta * n2 / n
    M2 = M2_1 + M2_2 + delta**2 * n1 * n2 / n

    return n, mean, M2

In [65]:
%%time
n, mean, M2 = reduce(reducer, map(mapper, Path('imdb-user-reviews').glob('**/*')))

# Check if n is greater than zero before calculating mean and standard deviation
if n > 0:
    print("Average rating:", mean)
    print("Standard deviation:", (M2 / n) ** 0.5)
else:
    print("No valid ratings found.")

Skipping file imdb-user-reviews/Forrest Gump/movieReviews.csv due to error: Expecting value: line 1 column 1 (char 0)
Processing file: imdb-user-reviews/Forrest Gump/metadata.json
Data: {'title': 'Forrest Gump', 'movieIMDbRating': 8.8, 'totalRatingCount': 2016919, 'totalUserReviews': '2.9K', 'totalCriticReviews': '173', 'metaScore': '82', 'movieGenres': ['Drama', 'Romance'], 'directors': ['Robert Zemeckis'], 'datePublished': '1994-07-06', 'creators': ['Winston Groom', 'Eric Roth'], 'mainStars': ['Tom Hanks', 'Robin Wright', 'Gary Sinise'], 'description': 'The presidencies of Kennedy and Johnson, the Vietnam War, the Watergate scandal and other historical events unfold from the perspective of an Alabama man with an IQ of 75, whose only desire is to be reunited with his childhood swe...', 'duration': 142}
Found rating: 8.8
Skipping file imdb-user-reviews/Pulp Fiction/movieReviews.csv due to error: Expecting value: line 1 column 1 (char 0)
Processing file: imdb-user-reviews/Pulp Fiction/m